In [ ]:
#Note, we originally ran this code in a separate file. We then transferred over the imporant code to this file. 
#We did not run it again because we did not want to get charged for API usage and were concerned we may go over the 
#limit of what's free if we doubled our total number of requests

In [ ]:
import pandas as pd

#import in the scraped data of schools we have
df_MA = pd.read_csv('massachusetts.csv')
df_VT = pd.read_csv('vermont.csv')
df_CT = pd.read_csv('connecticut.csv')
df_RI = pd.read_csv('rhode_island.csv')

HS_MA = ""
HS_VT = ""
HS_CT = ""
HS_RI = ""

#Google Maps API allows "Distance Matrices", so long as there are pipes between each location
for ad in df_MA["Address"]:
    HS_MA = HS_MA + ad+ "|"
    
for ad in df_VT["Address"]:
    HS_VT = HS_VT + ad+ "|"
    
for ad in df_CT["Address"]:
    HS_CT = HS_CT + ad+ "|"
    
for ad in df_RI["Address"]:
    HS_RI = HS_RI + ad+ "|"
    
#remove the final pipe character from each string
HS_MA = HS_MA[0:-1]
HS_VT = HS_VT[0:-1]
HS_CT = HS_CT[0:-1]
HS_RI = HS_RI[0:-1]

In [ ]:
import urllib.request as urllib
import pandas as pd
from bs4 import BeautifulSoup 

df_colleges = pd.DataFrame(columns = ["College", "City"])

#retrieve a list of colleges from New England

soup = BeautifulSoup(urllib.urlopen('https://nces.ed.gov/collegenavigator/?s=CT+ME+MA+NH+RI+VT&ic=1&pg=0').read())
for link in soup.find_all("strong"):
    if link.text =="U.S. Department of Education" or link.text =="13" or link.text =="12" or link.text == "11" or link.text =="10" or link.text =="1" or link.text =="2" or link.text =="3" or link.text =="4" or link.text =="5" or link.text =="6" or link.text =="7" or link.text =="8" or link.text =="9" or link.text == "My Favorites" or link.text == "comparisons":
        pass
    else:
        tempdict= {"College": link.text, "City": link.parent.parent.text[len(link.text)::] }
        df_colleges = df_colleges.append(tempdict, ignore_index=True)

In [ ]:
pd.set_option('display.max_rows', None)

#break down the scraped list into unique towns, such that there's never more than one college per town

df_colleges_unique = df_colleges.copy()
seen_places = []
del_indexes = []
count = -1
for row in df_colleges_unique["City"]:
    count = count + 1
    if row not in seen_places:
        seen_places.append(row)
    else: 
        del_indexes.append(count)
for index in del_indexes:
    df_colleges_unique = df_colleges_unique.drop(index)
df_colleges_unique=df_colleges_unique.reset_index()

In [ ]:
collegeaddstringMA = ""
collegeaddstringVT = ""
collegeaddstringRI = ""
collegeaddstringCT = ""

# creates four individual sets of the colleges, separated by state
# Maine and New Hampshire not included because high schools did not have enough data

for ind in range(0, len(df_colleges_unique["College"])):
    if "Massachusetts" in df_colleges_unique["City"][ind]:
        collegeaddstringMA = collegeaddstringMA + df_colleges_unique["College"][ind]+ "|"
    elif "Vermont" in df_colleges_unique["City"][ind]:
        collegeaddstringVT = collegeaddstringVT + df_colleges_unique["College"][ind]+ "|"
    elif "Rhode Island" in df_colleges_unique["City"][ind]:
        collegeaddstringRI = collegeaddstringRI + df_colleges_unique["College"][ind]+ "|"
    elif "Connecticut" in df_colleges_unique["City"][ind]:
        collegeaddstringCT = collegeaddstringCT + df_colleges_unique["College"][ind]+ "|"
    
collegeaddstringMA = collegeaddstringMA[0:-1]
collegeaddstringVT = collegeaddstringVT[0:-1]
collegeaddstringRI = collegeaddstringRI[0:-1]
collegeaddstringCT = collegeaddstringCT[0:-1]

In [ ]:
# helper function that takes a high school or college string for a state and splits into groups of ten
# Google Maps API cannot return a matrix with more than 100 elements at once

def splitby10(collegelist):
    collgrid = []
    for x in range((len(collegelist)//10)+1):
        collstring = ""
        i = x*10
        while i <= min((x*10)+9, len(collegelist)-1):
            collstring = collstring + str(collegelist[i])+"|"
            i = i+1
        collstring = collstring[0:-1]
        collgrid.append(collstring)
        
    return collgrid

In [ ]:
HS_MA_split=splitby10(HS_MA.split("|"))
HS_VT_split=splitby10(HS_VT.split("|"))
HS_CT_split=splitby10(HS_CT.split("|"))
HS_RI_split=splitby10(HS_RI.split("|"))
collegeMA_split = splitby10(collegeaddstringMA.split("|"))
collegeVT_split = splitby10(collegeaddstringVT.split("|"))
collegeCT_split = splitby10(collegeaddstringCT.split("|"))
collegeRI_split = [collegeaddstringRI] #less than 10 colleges in RI, but wanted to return a list

In [ ]:
import requests, json 
 
api_key ='my_api_key'
#we didn't want to put our API key on github- please let us know if you need to see it

distance_dict = {}

# the below was ran four times, each time with a different state
# running it in a loop with all four states is possible, but takes an extremely long amount of time
# it's also not efficient, given that we want to keep the CSVs separate until it's all over

for x in range(len(HS_VT_split)): #runs the high schools 10 at a time
    small_dict=None
    source = HS_VT_split[x]
    for i in range(len(collegeVT_split)): #for a set of 10 high schools, runs 10 colleges at a time
        dest = collegeVT_split[i]
        url ='https://maps.googleapis.com/maps/api/distancematrix/json?'
        r = requests.get(url + 'origins=' + source +
                   '&destinations=' + dest +
                   '&key=' + api_key)
        mat = r.json()
        rows = mat["rows"]
        if small_dict: #keeps track of the distances for every high school
            for f in range(len(HS_VT_split[x].split("|"))):
                small_dict[f]["elements"].extend(rows[f]["elements"])
        else:
            small_dict=rows
                
    l = []

    for e in small_dict:
        distance = 100000000 #arbitrarily large number that won't be surpassed
        for d in e["elements"]:
            if d["distance"]["value"] < distance:
                distance = d["distance"]["value"]
        l.append(distance)
    if distance_dict.get("Distance from Nearest College"):
        distance_dict["Distance from Nearest College"].extend(l)
    else:
        distance_dict["Distance from Nearest College"] = l

In [ ]:
df_VT["Distance from Nearest College"]=distance_dict["Distance from Nearest College"] #add to dataframe

In [ ]:
#We noticed that a few of our addresses were duplicates. We removed the duplicate rows here
import pandas as pd
df_VT = df_VT.drop_duplicates(subset="Address")
df_VT = df_VT.reset_index(drop=True)
df_VT = df_VT.drop(["Unnamed: 0"], axis=1)

In [ ]:
#Finally, we sent the final results from each state to a csv (in GitHub and imported in our fp4 file)
df_VT.to_csv('ver_with_dist.csv')